In [ ]:
from src.requestcompletion.llm.models._litellm_wrapper import LiteLLMWrapper
from src.requestcompletion.llm import ToolCall, Tool, Parameter
from src.requestcompletion.llm.message import UserMessage, SystemMessage, AssistantMessage
from src.requestcompletion.llm.history import MessageHistory
from pydantic import BaseModel
import litellm
from typing import List


In [ ]:
# Initialize the LiteLLMWrapper with the desired model
# litellm_model = LiteLLMWrapper(model_name="anthropic/claude-3-5-sonnet-20241022")
litellm_model = LiteLLMWrapper(model_name="openai/gpt-4o")

## General chat

In [ ]:
# Create a message history
messages = MessageHistory([
    SystemMessage("You are a helpful assistant."),
    UserMessage("Can you tell me a joke?")
])

# Perform a chat completion
response = litellm_model.chat(messages)

# Print the assistant's response
print(response.message.content)

## Structured output

In [ ]:
class DishInfo(BaseModel):
    name: str
    cuisine: str
    calories: int

messages = MessageHistory([
    SystemMessage("You are a nutrition expert"),
    UserMessage("Tell me about a popular Italian dish."),
])

result = litellm_model.structured(messages, DishInfo)

In [ ]:
result.message.content

In [ ]:
class Engine(BaseModel):
    manufacturing_country: str
    number_of_cylinders: int

class CarInfo(BaseModel):
    brand: str
    country: str
    engine: Engine

class Info(BaseModel):
    cars: List[CarInfo]

sports_cars_description = """
Among the world’s most thrilling sports cars, the Ferrari 812 Superfast, crafted in Italy,
features a naturally aspirated V12 engine also built in Italy, delivering blistering power
through its 12-cylinder setup. From Germany, the Porsche 911 Turbo S stands out with its
twin-turbocharged flat-six engine, manufactured in Germany, offering a perfect balance of
performance and refinement. Meanwhile, the Chevrolet Corvette Z06, proudly American-made
in the USA, houses a hand-built 5.5-liter flat-plane crank V8 engine, also produced in the
USA, boasting 8 cylinders of raw muscle. The McLaren 720S, a British marvel from the UK,
is equipped with a twin-turbo V8 engine made in England, showcasing 8 cylinders of
precision engineering. Finally, the Toyota GR Supra, originating from Japan, uses a
3.0-liter inline-6 engine built in Austria by BMW, blending Japanese design with German
power and offering 6 cylinders of smooth performance.
"""

messages = MessageHistory(
    [
        SystemMessage("You are a car enthusiast who can extract information from paragraphs about different cars."\
                                  "Ensure you analyze the whole paragraph and return information about all cars mentioned"),
        UserMessage(f"Extract the information about all of the cars mentioned in: \n {sports_cars_description}")
    ]
)

result = litellm_model.structured(messages, Info)

In [ ]:
result.message.content

## Stream chat

In [ ]:
messages = MessageHistory([
    UserMessage("Can you tell me a joke?")
])

resp = litellm_model.stream_chat(messages)

In [ ]:
resp.streamer

In [ ]:
for x in resp.streamer:
    print(x)


## Tool Calling

### Tools



In [ ]:
def available_locations() -> List[str]:
    """Returns a list of available locations.
    Args:
    Returns:
        List[str]: A list of available locations.
    """
    return [
        "New York",
        "Los Angeles",
        "Chicago",
        "Delhi",
        "Mumbai",
        "Bangalore",
        "Paris",
        "Denmark",
        "Sweden",
        "Norway",
        "Germany",
        "Vancouver",
        "Toronto",
    ]

def currency_used(location: str) -> str:
    """Returns the currency used in a location.
    Args:
        location (str): The location to get the currency used for.
    Returns:
        str: The currency used in the location.
    """
    currency_map = {
        "New York": "USD",
        "Los Angeles": "USD",
        "Chicago": "USD",
        "Delhi": "INR",
        "Mumbai": "INR",
        "Bangalore": "INR",
        "Paris": "EUR",
        "Denmark": "EUR",
        "Sweden": "EUR",
        "Norway": "EUR",
        "Germany": "EUR",
        "Vancouver": "CAD",
        "Toronto": "CAD",
    }
    used_currency = currency_map.get(location)
    if used_currency is None:
        raise ValueError(f"Currency not available for location: {location}")
    return used_currency

def average_location_cost(location: str, num_days: int) -> float:
    """Returns the average cost of living in a location for a given number of days.
    Args:
        location (str): The location to get the cost of living for.
        num_days (int): The number of days for the trip.
    Returns:
        float: The average cost of living in the location.
    """
    daily_costs = {
        "New York": 200.0,
        "Los Angeles": 180.0,
        "Chicago": 150.0,
        "Delhi": 50.0,
        "Mumbai": 55.0,
        "Bangalore": 60.0,
        "Paris": 220.0,
        "Denmark": 250.0,
        "Sweden": 240.0,
        "Norway": 230.0,
        "Germany": 210.0,
        "Vancouver": 200.0,
        "Toronto": 180.0,
    }
    daily_cost = daily_costs.get(location)
    if daily_cost is None:
        raise ValueError(f"Cost information not available for location: {location}")
    return daily_cost * num_days

def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
    """Converts currency using a static exchange rate (for testing purposes).
    Args:
        amount (float): The amount to convert.
        from_currency (str): The currency to convert from.
        to_currency (str): The currency to convert to.
    Returns:
        float: The converted amount.
    Raises:
        ValueError: If the exchange rate is not available.
    """
    exchange_rates = {
        ("USD", "EUR"): 0.85,
        ("EUR", "USD"): 1.1765,
        ("USD", "INR"): 83.0,
        ("INR", "USD"): 0.01205,
        ("EUR", "INR"): 98.0,
        ("INR", "EUR"): 0.0102,
        ("CAD", "USD"): 0.78,
        ("USD", "CAD"): 1.28,
        ("CAD", "EUR"): 0.66,
        ("EUR", "CAD"): 1.52,
        ("INR", "CAD"): 0.0125,
        ("CAD", "INR"): 80.0,
    }

    rate = exchange_rates.get((from_currency, to_currency))
    if rate is None:
        raise ValueError("Exchange rate not available")
    return amount * rate

### LiteLLM

In [ ]:
import json

# model = "gpt-3.5-turbo-1106"
# model = "gpt-4o"
model = "claude-3-sonnet-20240229"

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    elif "new york" in location.lower():
        return json.dumps({"location": "New York", "temperature": "75", "unit": "fahrenheit"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


def average_location_cost(location: str, num_days: int) -> float:
    """Returns the average cost of living in a location for a given number of days.
    Args:
        location (str): The location to get the cost of living for, e.g. "New York"
        num_days (int): The number of days for the trip.
    Returns:
        float: The average cost of living in the location.
    """
    daily_costs = {
        "New York": 200.0,
        "Los Angeles": 180.0,
        "Chicago": 150.0,
        "Delhi": 50.0,
        "Mumbai": 55.0,
        "Bangalore": 60.0,
        "Paris": 220.0,
        "Denmark": 250.0,
        "Sweden": 240.0,
        "Norway": 230.0,
        "Germany": 210.0,
        "Vancouver": 200.0,
        "Toronto": 180.0,
    }
    daily_cost = daily_costs.get(location)
    if daily_cost is None:
        raise ValueError(f"Cost information not available for location: {location}")
    return daily_cost * num_days

def test_parallel_function_call():
    try:
        # Step 1: send the conversation and available functions to the model
        messages = [{"role": "user", "content": "Give me a summary for 2 day trip to New York. Use NewYork as an arg if ou are tool_calling"}]
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "average_location_cost",
                    "description": "Get the average cost of living in a location for a given number of days",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The location to get the cost of living for.",
                            },
                            "num_days": {"type": "integer", "description": "The number of days for the trip"},
                        },
                        "required": ["location", "num_days"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "Get the current weather in a given location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                        },
                        "required": ["location"],
                    },
                },
            }
        ]
        response = litellm.completion(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",  # auto is default, but we'll be explicit
        )
        print("\nFirst LLM Response:\n", response)
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        print("\nLength of tool calls", len(tool_calls))

        # Step 2: check if the model wanted to call a function
        if tool_calls:
            # Step 3: call the function
            # Note: the JSON response may not always be valid; be sure to handle errors
            available_functions = {
                "get_current_weather": get_current_weather,
                "average_location_cost": average_location_cost,
            }  
            messages.append(response_message)  # extend conversation with assistant's reply

            # Step 4: send the info for each function call and function response to the model
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                print(function_args)
                function_response = function_to_call(**function_args)
                print(function_response)

                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": json.dumps(function_response),
                    }
                )  # extend conversation with function response

            print(messages)
            second_response = litellm.completion(
                model=model,
                messages=messages,
            )  # get a new response from the model where it can see the function response
            print("\nSecond LLM response:\n", second_response)
            return second_response
    except Exception as e:
      print(f"Error occurred: {e}")

x = test_parallel_function_call()

In [ ]:
tool_instances = [
    Tool(
        name="average_location_cost",
        detail="Get the average cost of living in a location for a given number of days",
        parameters={
            Parameter(
                name="location",
                param_type="string",
                description="The location to get the cost of living for.",
                required=True
            ),
            Parameter(
                name="num_days",
                param_type="integer",
                description="The number of days for the trip",
                required=True
            )
        }
    ),
    Tool(
        name="get_current_weather",
        detail="Get the current weather in a given location",
        parameters={
            Parameter(
                name="location",
                param_type="string",
                description="The city and state, e.g. San Francisco, CA",
                required=True
            ),
            Parameter(
                name="unit",
                param_type="string",
                description="The unit to use for temperature (celsius or fahrenheit)",
                required=False
            )
        }
    )
]

In [ ]:
messages = MessageHistory([SystemMessage("You are a travel agent with access to the following tools: get_current_weather, avergae_location_cost"),
                           UserMessage("Give me a summary for 2 day trip to New York. Use NewYork as an arg if ou are tool_calling"),
                          ])
response = litellm_model.chat_with_tools(messages, tool_instances)
messages.append(response.message)

In [ ]:
def _to_litellm_message(msg):
    """
    Convert your Message (UserMessage, AssistantMessage, ToolMessage) into
    the simple dict format that litellm.completion expects.
    """
    base = {"role": msg.role, "content": msg.content}
    return base

messages = [_to_litellm_message(m) for m in messages]

In [ ]:
print("\nFirst LLM Response:\n", response)
response_message = response.message.content
tool_calls = [tool_call for tool_call in response_message if isinstance(tool_call, ToolCall)]

print("\nLength of tool calls", len(tool_calls))

if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
        "average_location_cost": average_location_cost,
    }  
    messages.append(response_message)  # extend conversation with assistant's reply

    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.name
        function_to_call = available_functions[function_name]
        print(tool_call.arguments)
        function_response = function_to_call(**tool_call.arguments)

        messages.append(
            {
                "tool_call_id": tool_call.identifier,
                "role": "tool",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )  # extend conversation with function response


In [ ]:
messages

In [ ]:
second_response = litellm.completion(
    model=model,
    messages=messages,
)  # get a new response from the model where it can see the function response
print("\nSecond LLM response:\n", second_response)

# LiteLLM integrated RC

In [1]:
import requestcompletion as rc

terminal = rc.library.terminal_llm(pretty_name="Terminal",
                                   model=rc.llm.OpenAILLM("gpt-4o"),
                                   system_message=SystemMessage("You are a helpful assistant."),
                                   )


NameError: name 'SystemMessage' is not defined

In [ ]:
result = await rc.call(terminal, message_history=MessageHistory([UserMessage("How are you doing?")]))